### Iterative Quantum Amplitude Estimation

In [11]:
import numpy as np

def find_next_k(k_previous, l, u, up_previous, r =2):
    K_current = 4* k_previous + 2
    theta_min = K_current * l
    theta_max = K_current * u
    K_max = np.floor(np.pi / (u - l))
    K = K_max - (K_max -2 ) % 4

    while K > r* K_current:
        q = K/K_current
        if ((q* theta_max) % 2*np.pi) <= np.pi and ((q * theta_min) % 2*np.pi) <= np.pi:
            K_next = K
            up_next = True
            k_next = (K_next - 2) / 4
            return (k_next, up_next)
        if ((q* theta_max) % 2*np.pi) >= np.pi and ((q * theta_min) % 2*np.pi) >= np.pi:
            K_next = K
            up_next = False
            k_next = (K_next - 2) / 4
            return (k_next, up_next)
        K = K - 4
    return (k_previous, up_previous)


def iqae(epsilon, alpha, Nshots: int, ci = "Chernoff Hoeffding"):
    def cal_lmax(N_shots, alpha, T):
        return np.arcsin(2/N_shots*np.log(2*T/alpha)) ** (1/4)
    i = 0
    k_current = 0
    up_current = True
    [ltheta, utheta] = [0, np.pi/2]
    T = np.ceil(np.log2(np.pi/(8*epsilon)))
    L_max = cal_lmax(Nshots, alpha, T)

    while utheta - ltheta > 2*epsilon:
        i += 1
        k_previous = k_current
        up_previous = up_current
        k_current, up_current = find_next_k(k_previous, ltheta, utheta, up_previous)
        K_current = 4 * k_current + 2
        if K_current > np.ceil(L_max / epsilon):
            N = np.ceil(Nshots * L_max / epsilon / K_current / 10)
        else:
            N = Nshots

        # apply grover oraacle k_current times and measuring N times and get the value of a
        a = 0

        if k_current == k_previous:
            # calculate the results of all the iterations j<= i
            # with kj = ki into a single results\
            k_final = k_current
        
        if ci == "Chernoff Hoeffding":
            epsilon_hw = np.sqrt(1/2*N * np.log(2*T/alpha))
            a_max = np.min(1, a + epsilon_hw)
            a_min = np.max(0, a - epsilon_hw)

        theta_min = np.arcos(1 - 2*a_min) / K_current
        theta_max = np.arcos(1 - 2*a_max) / K_current

        ltheta = (np.floor(K_current * ltheta) % 2*np.pi + theta_min) / K_current
        utheta = (np.floor(K_current * utheta) % 2*np.pi + theta_max) / K_current
    [a_l, a_u] = [np.sin(ltheta) ** 2, np.sin(utheta) **2 ]
    return [a_l, a_u]

